In [1]:
%gui qt
import sys
import numpy as np
import pyqtgraph as pg
from pyqtgraph.Qt import QtGui, QtCore
from pyqtgraph.Point import Point
import pysiglabui

In [17]:
def PYSIGLAB(data):
    """
        SIGLAB : Born out of frustration with the you-know-who
    """
    app_created = False
    app = QtCore.QCoreApplication.instance()
    if app is None:
        app = QtGui.QApplication(sys.argv)
        app_created = True
    app.references = set()
    
    
    w = QtGui.QMainWindow()
    ui = pysiglabui.Ui_MainWindow()
    ui.setupUi(w)
    
    ui.spinBox_minVal.setMinimum(0)
    ui.spinBox_maxVal.setMaximum(1)
    ui.spinBox_minVal.setMaximum(data.size-1)
    ui.spinBox_maxVal.setMaximum(data.size)
    
    w.setWindowTitle('SIGLAB')
    w.resize(800, 600)
    w.show()
    
#     #generate layout
    view = ui.graphicsView
    
    p1 = view.addPlot(row=1, col=0)
    p2 = view.addPlot(row=2, col=0)

    region = pg.LinearRegionItem(brush=(0, 255, 0, 127))
    region.setZValue(10)
    region.setRegion([0, 1000])
    
    p2.addItem(region, ignoreBounds=True)
    p1.setAutoVisible(y=True)
    
    p1.plot(data, pen="b")

    p2.plot(data, pen="w")

    def update():
        region.setZValue(20)
        minX, maxX = region.getRegion()
        ui.spinBox_minVal.setValue(minX)
        ui.spinBox_maxVal.setValue(maxX)
        p1.setXRange(minX, maxX, padding=0)

    region.sigRegionChanged.connect(update)

    def updateRegion(window, viewRange):
        rgn = viewRange[0]
        region.setRegion(rgn)
    
    def updateRegionSpin():
        rgn = [ui.spinBox_minVal.value(), ui.spinBox_maxVal.value()]
        region.setRegion(rgn)
    

    p1.sigRangeChanged.connect(updateRegion)
    
    ui.spinBox_minVal.valueChanged.connect(updateRegionSpin)
    ui.spinBox_maxVal.valueChanged.connect(updateRegionSpin)
    
#     def b1_clicked(evt):
#         print "you clicked Clear"
#         minX, maxX = region.getRegion()
#         text, ok = QtGui.QInputDialog.getText(w, 'Input Dialog', 'Enter your name:', 'max:')
    
#     b1.clicked.connect(b1_clicked)
    
#     def b2_clicked(evt):
#         print "you clicked Save"
        
#     b2.clicked.connect(b2_clicked)

    if app_created:
        app.exec_()
    return w

In [18]:
data = 10000 + 15000 * pg.gaussianFilter(np.random.random(size=10000), 10) + 3000 * np.random.random(size=10000)

win = PYSIGLAB(data)